In [21]:
from matplotlib import pyplot as plt
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import numpy as np
from PIL import Image
import os

from os import path

lenet = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, 10))


In [23]:
ckpt = torch.load('../model/CNN_model_weight2.pth.tar')
lenet.load_state_dict(ckpt['state_dict'])            #参数加载到指定模型cnn

dir_all = '../mydigit/all/'
picList = []

for filename in os.listdir(dir_all):              #listdir的参数是文件夹的路径
    eachpic_name = dir_all+filename
    im = Image.open(eachpic_name).resize((28, 28))     #取图片数据
    im = im.convert('L')      #灰度图
    im_data = np.array(im)
    im_data = torch.from_numpy(im_data).float()
    picList.append(im_data)


计算准确率

In [24]:
correctTable = np.zeros((10,5))
n = 0
for index,value in enumerate(picList):
    value = value.reshape([1, 1, 28, 28])
    out = lenet(value)
    _, pred = torch.max(out, 1)
    pred = pred.numpy().tolist()[0]
    numLable = index//5
    correctTable[numLable][n] = (pred==numLable)
    n = (n+1)%5
correct_sum = np.sum(correctTable)
print("wrong:",50-correct_sum)
print("accuracy:",correct_sum/50*100,"%")


wrong: 11.0
accuracy: 78.0 %


显示图片，预测

In [ ]:
for eachpic in picList:              #listdir的参数是文件夹的路径
    plt.figure()
    plt.imshow(eachpic)
    plt.show()

    eachpic = eachpic.view(1, 1, 28, 28)
    out = lenet(eachpic)
    _, pred = torch.max(out, 1)

    print('predict:{}。'.format(pred[0]))